# Week 3 Assignment - Segmenting & Clustering Neighbourhoods in Toronto

## Part 1: Scraping Toronto Postal Codes

In [1]:
import pandas as pd
import numpy as np

### Storing Table Data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

webpage = pd.read_html(url)

df = webpage[0]

### Parsing Table Data

In [3]:
# Dropping postal codes that do not have an assigned borough
df = df[df.Borough != 'Not assigned']

# Sorting results & resetting the index
df = df.sort_values(['Postal Code'])
df = df.reset_index(drop = True)

In [4]:
# Checking if any cells have a borough but a Not Assigned neighbourhood
df[df['Neighbourhood'] == 'Not assigned']

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []

In [5]:
#Print table
df

Postal Code      Borough  \
0           M1B  Scarborough   
1           M1C  Scarborough   
2           M1E  Scarborough   
3           M1G  Scarborough   
4           M1H  Scarborough   
..          ...          ...   
98          M9N         York   
99          M9P    Etobicoke   
100         M9R    Etobicoke   
101         M9V    Etobicoke   
102         M9W    Etobicoke   

                                         Neighbourhood  
0                                       Malvern, Rouge  
1               Rouge Hill, Port Union, Highland Creek  
2                    Guildwood, Morningside, West Hill  
3                                               Woburn  
4                                            Cedarbrae  
..                                                 ...  
98                                              Weston  
99                                           Westmount  
100  Kingsview Village, St. Phillips, Martin Grove ...  
101  South Steeles, Silverstone, Humbergate, Jamest...  
102                Northwest, West Humber - Clairville  

[103 rows x 3 columns]

In [6]:
# Number of rows & columns of the dataframe
print(df.shape)

(103, 3)


## Part 2: Appending Geographical Coordinates

In [7]:
# Reading geospatial coordinates data
coord = pd.read_csv('http://cocl.us/Geospatial_data')

coord.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [8]:
# Merging the scraped data with the geospatial data
df_comb = pd.merge(df, coord)

df_comb.head()

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

## Part 3: Clustering Neighbourhoods

In [9]:
import matplotlib as mpl
from sklearn.cluster import KMeans
import folium
import requests
from pandas.io.json import json_normalize

### First, a map of the neighbourhoods in the above dataset

In [10]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df_comb['Latitude'], df_comb['Longitude'], df_comb['Borough'], df_comb['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Defining Foursquare credentials

In [11]:
CLIENT_ID = 'QN1I42SPFQLIPBYB1FD4NYQRF0NG0KDJVP5ZK5VSVHX3LZIF' # your Foursquare ID
CLIENT_SECRET = 'LKJNAME1PYZYY30ST2WA5GOX4J43ZBIYMVSNQV0JXTV44D5M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: QN1I42SPFQLIPBYB1FD4NYQRF0NG0KDJVP5ZK5VSVHX3LZIF
CLIENT_SECRET:LKJNAME1PYZYY30ST2WA5GOX4J43ZBIYMVSNQV0JXTV44D5M


### Defining function to get nearby venues in each Toronto neighbourhood

In [12]:
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Running the above function for Toronto neighbourhood

In [13]:
toronto_venues = getNearbyVenues(names = df_comb['Neighbourhood'],
                                 latitudes = df_comb['Latitude'],
                                 longitudes = df_comb['Longitude']
                                )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(2140, 7)


Neighbourhood  Neighbourhood Latitude  \
0                          Malvern, Rouge               43.806686   
1  Rouge Hill, Port Union, Highland Creek               43.784535   
2       Guildwood, Morningside, West Hill               43.763573   
3       Guildwood, Morningside, West Hill               43.763573   
4       Guildwood, Morningside, West Hill               43.763573   

   Neighbourhood Longitude                  Venue  Venue Latitude  \
0               -79.194353                Wendy’s       43.807448   
1               -79.160497  Royal Canadian Legion       43.782533   
2               -79.188711         RBC Royal Bank       43.766790   
3               -79.188711      G & G Electronics       43.765309   
4               -79.188711             Sail Sushi       43.765951   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.191151                  Bank  
3       -79.191537     Electronics Store  
4       -79.191275            Restaurant

### Grouping venues by neighbourhood

In [15]:
toronto_venues.groupby('Neighbourhood').count()

Neighbourhood Latitude  \
Neighbourhood                                                             
Agincourt                                                             4   
Alderwood, Long Branch                                               10   
Bathurst Manor, Wilson Heights, Downsview North                      20   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    25   
...                                                                 ...   
Willowdale, Willowdale East                                          32   
Willowdale, Willowdale West                                           5   
Woburn                                                                3   
Woodbine Heights                                                      6   
York Mills West                                                       4   

                                                 Neighbourhood Longitude  \
Neighbourhood                                                              
Agincourt                                                              4   
Alderwood, Long Branch                                                10   
Bathurst Manor, Wilson Heights, Downsview North                       20   
Bayview Village                                                        4   
Bedford Park, Lawrence Manor East                                     25   
...                                                                  ...   
Willowdale, Willowdale East                                           32   
Willowdale, Willowdale West                                            5   
Woburn                                                                 3   
Woodbine Heights                                                       6   
York Mills West                                                        4   

                                                 Venue  Venue Latitude  \
Neighbourhood                                                            
Agincourt                                            4               4   
Alderwood, Long Branch                              10              10   
Bathurst Manor, Wilson Heights, Downsview North     20              20   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   25              25   
...                                                ...             ...   
Willowdale, Willowdale East                         32              32   
Willowdale, Willowdale West                          5               5   
Woburn                                               3               3   
Woodbine Heights                                     6               6   
York Mills West                                      4               4   

                                                 Venue Longitude  \
Neighbourhood                                                      
Agincourt                                                      4   
Alderwood, Long Branch                                        10   
Bathurst Manor, Wilson Heights, Downsview North               20   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             25   
...                                                          ...   
Willowdale, Willowdale East                                   32   
Willowdale, Willowdale West                                    5   
Woburn                                                         3   
Woodbine Heights                                               6   
York Mills West                                                4   

                                                 Venue Category  
Neighbourhood                                                    
Agincourt                                                     4  
Alderwood, Long Branch                

### Breaking down venues by category

In [16]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# Move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# Grouping by neighbourhood - mean
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Neighbourhood  Accessories Store  \
0                                         Agincourt                0.0   
1                            Alderwood, Long Branch                0.0   
2   Bathurst Manor, Wilson Heights, Downsview North                0.0   
3                                   Bayview Village                0.0   
4                 Bedford Park, Lawrence Manor East                0.0   
..                                              ...                ...   
92                      Willowdale, Willowdale East                0.0   
93                      Willowdale, Willowdale West                0.0   
94                                           Woburn                0.0   
95                                 Woodbine Heights                0.0   
96                                  York Mills West                0.0   

    Afghan Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                 0.0      0.0                 0.0             0.0   
1                 0.0      0.0                 0.0             0.0   
2                 0.0      0.0                 0.0             0.0   
3                 0.0      0.0                 0.0             0.0   
4                 0.0      0.0                 0.0             0.0   
..                ...      ...                 ...             ...   
92                0.0      0.0                 0.0             0.0   
93                0.0      0.0                 0.0             0.0   
94                0.0      0.0                 0.0             0.0   
95                0.0      0.0                 0.0             0.0   
96                0.0      0.0                 0.0             0.0   

    Airport Service  Airport Terminal  American Restaurant  Antique Shop  ...  \
0               0.0               0.0                 0.00           0.0  ...   
1               0.0               0.0                 0.00           0.0  ...   
2               0.0               0.0                 0.00           0.0  ...   
3               0.0               0.0                 0.00           0.0  ...   
4               0.0               0.0                 0.04           0.0  ...   
..              ...               ...                  ...           ...  ...   
92              0.0               0.0                 0.00           0.0  ...   
93              0.0               0.0                 0.00           0.0  ...   
94              0.0               0.0                 0.00           0.0  ...   
95              0.0               0.0                 0.00           0.0  ...   
96              0.0               0.0                 0.00           0.0  ...   

    Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0             0.0                            0.0               0.0   
1             0.0                            0.0               0.0   
2             0.0                            0.0               0.0   
3             0.0                            0.0               0.0   
4             0.0                            0.0               0.0   
..            ...                            ...               ...   
92            0.0                            0.0               0.0   
93            0.0                            0.0               0.0   
94            0.0                            0.0               0.0   
95            0.0                            0.0               0.0   
96            0.0                            0.0               0.0   

    Vietnamese Restaurant  Warehouse Store  Wine Bar  Wine Shop  Wings Joint  \
0                 0.00000              0.0       0.0        0.0          0.0   
1                 0.00000              0.0       0.0        0.0          0.0   
2                 0.00000              0.0       0.0        0.0          0.0   
3                 0.00000              0.0       0.0        0.0          0.0   
4                 0.00000              0.0       0.0        0.0          0.0   
..            

### Top 5 most common venue categories by neighbourhood

In [17]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4         Miscellaneous Shop  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2             Pub   0.1
3             Gym   0.1
4  Sandwich Place   0.1


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2  Middle Eastern Restaurant  0.05
3              Shopping Mall  0.05
4          Mobile Phone Shop  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.08
1               R

### Storing most common venue categories by neighbourhood

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighbourhoods_venues_sorted.shape)
neighbourhoods_venues_sorted.head()

(97, 11)


Neighbourhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

  2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
0        Breakfast Spot  Latin American Restaurant          Skating Rink   
1              Pharmacy                Coffee Shop        Sandwich Place   
2                  Bank        Fried Chicken Joint         Shopping Mall   
3                  Bank         Chinese Restaurant   Japanese Restaurant   
4        Sandwich Place         Italian Restaurant            Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Donut Shop                 Diner        Discount Store   
1                   Pub          Dance Studio                  Pool   
2           Bridal Shop        Sandwich Place                 Diner   
3           Yoga Studio                 Diner        Discount Store   
4           Pizza Place              Boutique             Juice Bar   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0        Distribution Center               Dog Run       Doner Restaurant  
1               Skating Rink                   Gym             Donut Shop  
2  Middle Eastern Restaurant            Restaurant          Deli / Bodega  
3        Distribution Center               Dog Run       Doner Restaurant  
4       Fast Food Restaurant               Butcher                   Café

### Clustering neighbourhoods
Assumption of 10 clusters given large number of neighbourhoods
Would be preferable to optimise number of clusters based on elbow point of k vs average distance between points in each cluster

In [19]:
# Set number of clusters to 10
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_

array([9, 1, 1, 1, 1, 9, 1, 9, 1, 1, 0, 9, 9, 9, 1, 9, 1, 1, 9, 1, 1, 5,
       9, 1, 1, 9, 9, 9, 9, 9, 1, 9, 1, 9, 9, 9, 1, 1, 2, 4, 1, 1, 2, 9,
       9, 1, 9, 9, 9, 9, 7, 8, 1, 8, 1, 9, 9, 6, 9, 4, 9, 1, 9, 9, 9, 9,
       8, 1, 5, 1, 9, 1, 1, 9, 1, 1, 9, 9, 1, 1, 1, 1, 1, 1, 9, 1, 9, 3,
       1, 0, 9, 0, 1, 1, 9, 9, 0], dtype=int32)

### Adding cluster labels to the dataframe

In [20]:
kmeans_labels = kmeans.labels_.tolist()

labels = {'Neighbourhood':toronto_grouped[['Neighbourhood']],'Cluster Labels':kmeans_labels}

df_labels = pd.DataFrame(toronto_grouped[['Neighbourhood']])
df_labels['Cluster Labels'] = kmeans_labels

# Add cluster labels for each neighbourhood
toronto_labelled = pd.merge(df_labels, neighbourhoods_venues_sorted)

# Add coordinates for each neighbourhood
toronto_merged = pd.merge(df_comb, toronto_labelled)

toronto_merged.head()

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353               7  Fast Food Restaurant      Department Store   
1 -79.160497               5                   Bar           Yoga Studio   
2 -79.188711               9                  Bank        Breakfast Spot   
3 -79.216917               9           Coffee Shop     Korean Restaurant   
4 -79.239476               9      Hakka Restaurant                Bakery   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Event Space  Ethiopian Restaurant     Electronics Store   
1      Doner Restaurant    Dim Sum Restaurant                 Diner   
2     Electronics Store            Restaurant        Medical Center   
3             Drugstore                 Diner        Discount Store   
4  Caribbean Restaurant    Athletics & Sports   Fried Chicken Joint   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0  Eastern European Restaurant   Dumpling Restaurant             Drugstore   
1               Discount Store   Distribution Center               Dog Run   
2          Rental Car Location          Intersection    Mexican Restaurant   
3          Distribution Center               Dog Run      Doner Restaurant   
4              Thai Restaurant           Gas Station                  Bank   

  9th Most Common Venue 10th Most Common Venue  
0            Donut Shop       Doner Restaurant  
1            Donut Shop         Farmers Market  
2         Moving Target                  Diner  
3            Donut Shop            Yoga Studio  
4      Doner Restaurant                Dog Run

### Visualising the clusters

In [21]:
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Thank you!